# Задача 3.1


**[Выборка](https://github.com/natasha/nerus)**: набор предложений на русском языке с указанием частей речи для каждого слова.

Требуется:
1. Рассмотреть последовательность частей речи как марковскую модель. Определить оптимальный порядок марковской модели.
2. Обучить скрытую марковскую модель по выборке. Оценить точность предсказания частей речи, посчитать энтропию на выборке.

**Важно**: в целях ускорения эксперимента рекомендуется взять первые 10 МБ текста из выборки.


In [ ]:
import re

import numpy as np
import nltk
from nltk.tag import hmm
from nltk.corpus import brown
import pandas as pd

from collections import Counter

import nltk.lm as lm
from nltk.util import ngrams as nltk_ngrams
import numpy as np




In [8]:
!gunzip -c data/nerus_lenta.conllu.gz | head
!ls data/nerus_lenta.conllu

# newdoc id = 0
# sent_id = 0_0
# text = Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости.
1	Вице-премьер	_	NOUN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing	7	nsubj	_	Tag=O
2	по	_	ADP	_	_	4	case	_	Tag=O
3	социальным	_	ADJ	_	Case=Dat|Degree=Pos|Number=Plur	4	amod	_	Tag=O
4	вопросам	_	NOUN	_	Animacy=Inan|Case=Dat|Gender=Masc|Number=Plur	1	nmod	_	Tag=O
5	Татьяна	_	PROPN	_	Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing	1	appos	_	Tag=B-PER
6	Голикова	_	PROPN	_	Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing	5	flat:name	_	Tag=I-PER
7	рассказала	_	VERB	_	Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	Tag=O

gzip: stdout: Broken pipe
data/nerus_lenta.conllu


In [3]:
from nerus import load_nerus

docs = load_nerus("data/nerus_lenta.conllu.gz")

In [5]:
import tqdm

list_of_sent = []
list_of_toks = []

for i, doc in tqdm.tqdm(enumerate(docs)):
    for sent in doc.sents:
        list_of_toks_in_sent = []
        list_of_tags_in_sent = []
        for tok in sent.tokens:
            list_of_toks_in_sent.append(tok.text)
            list_of_tags_in_sent.append(tok.pos)
        list_of_sent.append(list_of_toks_in_sent)
        list_of_toks.append(list_of_tags_in_sent)
    if i > 10000:
        break

10001it [00:45, 219.64it/s]


### Определить оптимальный порядок марковской модели.

Добавим начальные и конечные токены.

In [6]:
list_of_toks_padded = [['<s>']+i+['<\s>'] for i in list_of_toks]

### Cловарь n-грамм и их частот и n-граммных префиксов и их частот

In [9]:
from collections import Counter

def ngrams_and_prefix_counts(s_tokens, n_max):
    ngrams_counts = {}
    prefix_counts = {}
    
    for i in range(n_max):
        ngrams_counts[i + 1] = Counter()
        prefix_counts[i + 1] = Counter()
        
    for sentence in s_tokens:
        n = len(sentence)
        for i in range(n_max):
            ngrams_counts[i + 1] += Counter([tuple(sentence[j : j + i + 1]) for j in range(n - i)])
            prefix_counts[i + 1] += Counter([tuple(sentence[j : j + i] + ['*']) for j in range(n - i)])

    return ngrams_counts, prefix_counts

In [10]:
ngrams_counts, prefix_counts = ngrams_and_prefix_counts(list_of_toks_padded, 3)

In [ ]:
prefix_counts

#### N-граммы и их частотные вероятности

$$\hat p_i = \hat p(w_i)$$

In [11]:
def unigram_probas(ngram_counts, prefix_counts):
    p1 = {}
    for w in ngram_counts[2]:
        pre_w = tuple([w[0]] + ['*'])
        p1[u'{0}'.format(*w)] = ngram_counts[2][w] / prefix_counts[2][pre_w]
    return p1

def bigram_probas(ngram_counts, prefix_counts):
    p2 = {}
    for w in ngram_counts[2]:
        pre_w = tuple([w[0]] + ['*'])
        p2[u'{1}|{0}'.format(*w)] = ngram_counts[2][w] / prefix_counts[2][pre_w]
    return p2

def trigram_probas(ngram_counts, prefix_counts):
    p3 = {}
    for w in ngram_counts[3]:
        pre_w = w[:2] + tuple(['*'])
        p3[u'{2}|{1},{0}'.format(*w)] = ngram_counts[3][w] / prefix_counts[3][pre_w]
    return p3

In [12]:
p1 = unigram_probas(ngrams_counts, prefix_counts)
p2 = bigram_probas(ngrams_counts, prefix_counts)
p3 = trigram_probas(ngrams_counts, prefix_counts)
p1

#### Проверка гипотезы, что триграммную модель можно свести к биграммной против правосторонней альтернативы

Статистика:
$$-2 \log (\prod_{i, j, k = 1}^m (\hat p_{ij} / \hat p_{ijk})^{n_{ijk}}) = \sum_{i, j, k}^m -2 n_{ijk} \log \hat p_{ij} + 2 n_{ijk} \log \hat p_{ijk} = \sum_{i = 3}^N -2 \log \hat p_{i,i - 1} + 2 \log \hat p_{i, i - 1, i - 2},$$
$$n_{ijk} = |\{X_t: X_t = O_i, X_{t + 1} = O_j, X_{t + 2} = O_k\}|$$

In [13]:
def chi2_statistic(p2, p3, tokens):
    stat2 = []
    stat3 = []
    for sent in tokens:
        n = len(sent)
        for i in range(n - 2):
            w = sent[i : i + 3]
            ngram3 = '{2}|{1},{0}'.format(*w)
            ngram2 = '{1}|{0}'.format(*w)
            stat2.append(np.clip(np.log(p2[ngram2]), np.log(1e-20), np.log(1e+20)))
            stat3.append(np.clip(np.log(p3[ngram3]), np.log(1e-20), np.log(1e+20)))
    return 2 * np.sum(stat3) - 2 * np.sum(stat2) 

def chi2_statistic_l(p1, p2, tokens):
    stat1 = []
    stat2 = []
    for sent in tokens:
        n = len(sent)
        for i in range(n - 2):
            w = sent[i : i + 2]
            ngram2 = '{1}|{0}'.format(*w)
            ngram1 = '{1}'.format(*w)
            stat2.append(np.clip(np.log(p2[ngram2]), np.log(1e-20), np.log(1e+20)))
            stat1.append(np.clip(np.log(p1[ngram1]), np.log(1e-20), np.log(1e+20)))
    return 2 * np.sum(stat2) - 2 * np.sum(stat1) 

In [17]:
import numpy as np
import scipy.stats as st

m = len(p3)
stat = chi2_statistic(p2, p3, list_of_toks_padded)

print(f'p-value = {1 - st.distributions.chi2(m * ((m - 1) ** 2) - 1).cdf(stat)}')

p-value = 1.0


Значит триграммная модель бесполезна.

In [18]:
m = len(p2)
stat = chi2_statistic_l(p1, p2, list_of_toks_padded)

print(f'p-value = {1 - st.distributions.chi2(m * ((m - 1) ** 2) - 1).cdf(stat)}')

p-value = 0.0


**А вот уже с биграмной так сделать нельзя — она полезна!**

### Соответственно, оптимальный порядок марковской модели = 2.

Проверим, что нам оно выдаст ([source](https://stackoverflow.com/questions/54959340/nltk-language-modeling-confusion)).

In [41]:
import nltk.lm as lm

mle = lm.MLE(2)
train, tr_vocab = lm.preprocessing.padded_everygram_pipeline(2, list_of_toks)
mle.fit(train, tr_vocab)
mle.generate(10)

['NOUN',
 'PUNCT',
 'SCONJ',
 'ADP',
 'ADJ',
 'NOUN',
 'VERB',
 'NOUN',
 'CCONJ',
 'VERB']

#### Вроде бы даже что-то разумное.

### Оценить точность предсказания частей речи, посчитать энтропию на выборке.

In [22]:
data = []
for i in range(len(list_of_toks)):
    for j in range(len(list_of_sent[i]))
        data.append[(list_of_sent[i][j], list_of_toks[i][j])]
    
data_train = data[ int(len(data)*0.1):]
data_test  = data[:int(len(data)*0.1) ]

In [26]:
from nltk.tag import hmm

tagger = hmm.HiddenMarkovModelTagger.train(data_train)
mean_entropy = 0  

for i in tqdm.tqdm(data_test):
    entropy = tagger.entropy(i)
    if entropy < np.inf:
        mean_entropy += entropy
        
mean_entropy /= len(data_test) 

print(f'Accuracy = {tagger.evaluate(data_test)}')
print(f'Mean entropy = {mean_entropy}')

100%|██████████| 11929/11929 [13:48<00:00, 14.40it/s]


Accuracy = 0.9484459872079195
Mean entropy = 6.539898218874228
